the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [ ]:
write_pretrain_data = False

import os
import shutil

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py
import math

dir = "/ext/Data/distracted_driver_detection/"

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    print(MODEL.__name__)
    gen = ImageDataGenerator()
    batch_size = 16
    train_generator = gen.flow_from_directory(os.path.join(dir, 'train'), image_size, shuffle=False, batch_size=batch_size)
    test_generator = gen.flow_from_directory(os.path.join(dir, 'test'), image_size, shuffle=False, batch_size=batch_size, class_mode=None)

    print("predict_generator train {}".format(math.ceil(train_generator.samples/batch_size)))n
    train = model.predict_generator(train_generator, math.ceil(train_generator.samples/batch_size))
    print("train: {}".format(train.shape))
    print("predict_generator test {}".format(math.ceil(test_generator.samples/batch_size)))
    test = model.predict_generator(test_generator, math.ceil(test_generator.samples/batch_size))
    print("test: {}".format(test.shape))
    print("label: {}".format(train_generator.classes.shape))

    print("begin create database {}".format(Model.__name__))
    with h5py.File("models/bottleneck_%s.h5" % MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
    print("write_gap {} successed".format(Model.__name__))
    
if write_pretrain_data:
    write_gap(ResNet50, (224, 224))
    write_gap(Xception, (299, 299))
    write_gap(InceptionV3, (299, 299))
    write_gap(VGG16, (224, 224))
    write_gap(VGG19, (224, 224))

In [27]:
import h5py
import numpy as np
from sklearn.utils import shuffle

np.random.seed(2017)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_test = np.array(h['test'])
        y_train = np.array(h['label'])
        X_train, y_train = shuffle(X_train, y_train)
        return X_train, y_train, X_test
    
def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)
    
from keras.models import *
from keras.layers import *

def make_model(input_shape):

#     input_tensor = Input(shape=input_shape)
#     #x = Flatten()(inp)
#     x = Dropout(0.5)(input_tensor)
#     x = Dense(10, activation='softmax')(x)
#     model = Model(input_tensor, x)
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    input_tensor = Input(input_shape)
    x = Dropout(0.5)(input_tensor)
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
print("Done")

Done


ResNet50

In [18]:
X_train, y_train, X_test = load_data("bottleneck_ResNet50.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.2)

load data: bottleneck_ResNet50.h5


ValueError: Error when checking target: expected dense_8 to have shape (None, 10) but got array with shape (22424, 1)

Xception

In [36]:
X_train, y_train, X_test = load_data("bottleneck_Xception.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=512, epochs=100, validation_split=0.2)

load data: bottleneck_Xception.h5
Train on 17939 samples, validate on 4485 samples
Epoch 1/100
17939/17939 [==============================] - 0s - loss: 2.0730 - acc: 0.2764 - val_loss: 1.7328 - val_acc: 0.5610
Epoch 2/100
17939/17939 [==============================] - 0s - loss: 1.6147 - acc: 0.5290 - val_loss: 1.4047 - val_acc: 0.7237
Epoch 3/100
17939/17939 [==============================] - 0s - loss: 1.3623 - acc: 0.6269 - val_loss: 1.2052 - val_acc: 0.7632

InceptionV3

In [37]:
X_train, y_train, X_test = load_data("bottleneck_InceptionV3.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=512, epochs=100, validation_split=0.2)

load data: bottleneck_InceptionV3.h5
Train on 17939 samples, validate on 4485 samples
Epoch 1/100
17939/17939 [==============================] - 0s - loss: 2.2584 - acc: 0.2103 - val_loss: 1.6174 - val_acc: 0.5572
Epoch 2/100
17939/17939 [==============================] - 0s - loss: 1.6567 - acc: 0.4317 - val_loss: 1.2605 - val_acc: 0.6981
Epoch 3/100
17939/17939 [==============================] - 0s - loss: 1.3642 - acc: 0.5488 - val_loss: 1.0645 - val_acc: 0.7527

VGG16

In [43]:
X_train, y_train, X_test = load_data("bottleneck_VGG16.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=20, validation_split=0.2)

load data: bottleneck_VGG16.h5
Train on 17939 samples, validate on 4485 samples
Epoch 1/20
17939/17939 [==============================] - 0s - loss: 9.7821 - acc: 0.1576 - val_loss: 3.1006 - val_acc: 0.4247
Epoch 2/20
17939/17939 [==============================] - 0s - loss: 5.5683 - acc: 0.3291 - val_loss: 0.9085 - val_acc: 0.7496

VGG19

In [47]:
X_train, y_train, X_test = load_data("bottleneck_VGG19.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.2)

load data: bottleneck_VGG19.h5
Train on 17939 samples, validate on 4485 samples
Epoch 1/30
17939/17939 [==============================] - 0s - loss: 8.7253 - acc: 0.1731 - val_loss: 1.6872 - val_acc: 0.5367
Epoch 2/30
17939/17939 [==============================] - 0s - loss: 4.3694 - acc: 0.3570 - val_loss: 0.7533 - val_acc: 0.7612

combine  ResNet50,Xception and InceptionV3

In [28]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["bottleneck_ResNet50.h5", "bottleneck_Xception.h5", "bottleneck_InceptionV3.h5"]:
    print('------------------'+filename)
    with h5py.File("models/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
print(X_train.shape)
print(y_train.shape)

------------------bottleneck_ResNet50.h5
------------------bottleneck_Xception.h5
------------------bottleneck_InceptionV3.h5
(22424, 6144)
(22424, 10)


In [29]:
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_split=0.2)

Train on 17939 samples, validate on 4485 samples
Epoch 1/30
17939/17939 [==============================] - 0s - loss: 0.2621 - acc: 0.9084 - val_loss: 0.1596 - val_acc: 0.9397

In [30]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
print()
print(y_pred[:1])

import pandas as pd
from keras.preprocessing.image import *

dir = "/ext/Data/distracted_driver_detection/"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(dir + "test/", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    l.append( [name, *y_pred[i]] )
    #index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    #df.set_value(index-1, 'label', y_pred[i])
    #df.append(df2, ignore_index=True
l = np.array(l)
data = {'img': l[:,0]}
for i in range(10):
    data["c%d"%i] = l[:,i+1]
df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
df.head(10)
df = df.sort_values(by='img')
df.to_csv('pred.csv', index=None, float_format='%.3f')

79200/79726 [============================>.] - ETA: 0s  